In [ ]:
import pandas as pd
import numpy as np

In [ ]:

import os
cols_to_read = {'CASE_STATUS': str, 'DECISION_DATE': str, 'JOB_TITLE': str, 'SOC_CODE': str, 'SOC_TITLE': str, 'FULL_TIME_POSITION': str, \
                'EMPLOYER_NAME': str, 'EMPLOYER_CITY': str, 'EMPLOYER_STATE': str, 'EMPLOYER_COUNTRY': str, 'WORKSITE_CITY': str, 'WORKSITE_STATE': str,  \
                'WAGE_RATE_OF_PAY_FROM': float, 'WAGE_UNIT_OF_PAY': str, 'PREVAILING_WAGE': float, 'PW_UNIT_OF_PAY': str, 'PW_WAGE_LEVEL': str, 'H_1B_DEPENDENT': str}


def read_lca_data(years, quarters):
    dfs = {}
    for year in years:
        for quarter in quarters:
            if os.path.exists(f'LCA_Disclosure_Data_FY{year}_Q{quarter}.parquet'):
                try:
                    df = pd.read_parquet(f'data/LCA_Disclosure_Data_FY{year}_Q{quarter}.parquet', columns=cols_to_read)
                    dfs[f'{year}_Q{quarter}'] = df
                    print(f"Successfully loaded LCA_Disclosure_Data_FY{year}_Q{quarter}.parquet with {len(df)} rows")
                except Exception as e:
                    print(f"An error occurred while reading LCA_Disclosure_Data_FY{year}_Q{quarter}.parquet: {e}")

            else:
                filename = f'data/LCA_Disclosure_Data_FY{year}_Q{quarter}.xlsx'
                try:
                    df = pd.read_excel(filename, usecols=cols_to_read.keys(), dtype=cols_to_read)
                    df.to_parquet(f'data/LCA_Disclosure_Data_FY{year}_Q{quarter}.parquet')
                    dfs[f'{year}_Q{quarter}'] = df
                    print(f"Successfully loaded {filename} with {len(df)} rows")
                except FileNotFoundError:
                    print(f"File {filename} not found. Skipping.")
                except Exception as e:
                    print(f"An error occurred while reading {filename}: {e}")
    return dfs


# Read data from all available years
dfs = read_lca_data([2022, 2023], [1, 2, 3, 4])

# Concatenate all dataframes vertically
df_combined = pd.concat([df for df in dfs.values()], axis=0, ignore_index=True)

print(f"\nTotal combined dataframe shape: {df_combined.shape}")
print(f"Number of datasets combined: {len(dfs)}")
print(f"Individual dataset sizes:")
for key, df in dfs.items():
    print(f"  {key}: {len(df)} rows")

In [ ]:
import requests

def get_lca_data(year, quarter):
    url = f"https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY{year}_Q{quarter}.xlsx"

    response = requests.get(url)
    with open(f'data/LCA_Disclosure_Data_FY{year}_Q{quarter}.xlsx', 'wb') as f:
        f.write(response.content)

for year in [2022, 2023]:
    for quarter in [1, 2, 3, 4]:
        get_lca_data(year, quarter)

!du -sh *.xlsx